In [1]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix,order_sample_pairs_by_distance


done1


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'01022018'

In [ ]:
#load the binary shared sequence table:

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan10_434Samples'
sharingMatrix_moreThan10_434Samples=pd.read_pickle(file1)

In [ ]:
sharingMatrix_moreThan10_434Samples.head()

In [ ]:
434*19477

In [ ]:
sharingMatrix_moreThan10_434Samples.sum().sum()

In [ ]:
416268./8453018

In [ ]:
sharingMatrix_moreThan10_434Samples['nSharedSeqsPerPerson']=sharingMatrix_moreThan10_434Samples.sum(axis=1)
sharingMatrix_moreThan10_434Samples.head()

In [ ]:
sharingMatrix_moreThan10_434Samples.loc['nSamplesSharingSeq',:]=sharingMatrix_moreThan10_434Samples.sum(axis=0)
sharingMatrix_moreThan10_434Samples.tail()

In [ ]:
sharingMatrix_moreThan10_434Samples.iloc[:-1,:]

In [ ]:
sharingMatrix_moreThan10_434Samples=sharingMatrix_moreThan10_434Samples.sort_values(by='nSharedSeqsPerPerson',
                                                                                ascending=False)
sharingMatrix_moreThan10_434Samples.head()

In [ ]:
sharingMatrix_moreThan10_434Samples=sharingMatrix_moreThan10_434Samples.sort_values(by='nSamplesSharingSeq',
                                                                                ascending=False,axis=1)
sharingMatrix_moreThan10_434Samples.head()

In [ ]:
sharingMatrix_moreThan10_434Samples.tail()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(sharingMatrix_moreThan10_434Samples.iloc[1:,0])
plt.title('sharing statistics-moreThan10binaryTable')
plt.xlabel('nSharedSeqsPerPerson')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(sharingMatrix_moreThan10_434Samples.iloc[0,1:],bins=20)
plt.title('sharing statistics-moreThan10binaryTable')
plt.xlabel('nSamplesSharingSeq')
plt.ylabel('Frequency')
plt.show()

In [ ]:
tmp=sharingMatrix_moreThan10_434Samples

In [ ]:
sharingMatrix_moreThan10_434Samples_sorted=sharingMatrix_moreThan10_434Samples.iloc[1:,1:]
sharingMatrix_moreThan10_434Samples_sorted_SHORT=sharingMatrix_moreThan10_434Samples_sorted.iloc[:,:1000]

In [ ]:
fig, axes = plt.subplots(figsize=(20,20))
axes.imshow(sharingMatrix_moreThan10_434Samples_sorted_SHORT, interpolation=None)
plt.show()

In [ ]:
sharingMatrix_moreThan10_434Samples_sorted_SHORT.head()

In [ ]:
#load the RA shared sequence table:

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan10_434Samples_RA'
sharingMatrix_moreThan10_434Samples_RA=pd.read_pickle(file1)

In [ ]:
sharingMatrix_moreThan10_434Samples_RA.head()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA['TotalFreqOfAllSharedSeqsPerSample']=sharingMatrix_moreThan10_434Samples_RA.sum(axis=1)
sharingMatrix_moreThan10_434Samples_RA.head()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA.loc['TotalFreqOfSeqOverAllSamples']=sharingMatrix_moreThan10_434Samples_RA.sum(axis=0)
sharingMatrix_moreThan10_434Samples_RA.tail()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA=sharingMatrix_moreThan10_434Samples_RA.sort_values(by='TotalFreqOfAllSharedSeqsPerSample',
                                                                                ascending=False)
sharingMatrix_moreThan10_434Samples_RA.head()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA=sharingMatrix_moreThan10_434Samples_RA.sort_values(by='TotalFreqOfSeqOverAllSamples',
                                                                                ascending=False,axis=1)
sharingMatrix_moreThan10_434Samples_RA.head()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA.tail()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(sharingMatrix_moreThan10_434Samples_RA.iloc[1:,0])
plt.title('sharing statistics-moreThan10RATable')
plt.xlabel('TotalFreqOfAllSharedSeqsPerSample')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(sharingMatrix_moreThan10_434Samples_RA.iloc[0,1:],bins=50)
plt.title('sharing statistics-moreThan10binaryTable')
plt.xlabel('TotalFreqOfSeqOverAllSamples')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sharingMatrix_moreThan10_434Samples_RA_head=sharingMatrix_moreThan10_434Samples_RA.iloc[1:,1:500]

In [ ]:
sharingMatrix_moreThan10_434Samples_RA_head.head()

In [ ]:
fig, axes = plt.subplots(figsize=(8,8))
axes.imshow(sharingMatrix_moreThan10_434Samples_RA_head, interpolation=None, vmin=0, vmax=0.01, cmap='PuRd')
plt.show()

# view distance matrix:

## binary:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan10_euclidean'
distMat_PNP434_sharingMatrixMoreThan10_euclidean=pd.read_pickle(file1)

In [ ]:
distMat_PNP434_sharingMatrixMoreThan10_euclidean.head()

In [ ]:
from skbio.stats.distance import DistanceMatrix


In [ ]:
import pandas as pd
from scipy.spatial import distance_matrix

In [ ]:
df=sharingMatrix_moreThan10_434Samples
newDistMat10binaryEuc=pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)


In [ ]:
newDistMat10binaryEuc.shape

In [ ]:
newDistMat10binaryEuc=newDistMat10binaryEuc.reindex_axis(newDistMat10binaryEuc.sum().sort_values().index, axis=1)
newDistMat10binaryEuc=newDistMat10binaryEuc.reindex_axis(newDistMat10binaryEuc.sum().sort_values().index, axis=0)
newDistMat10binaryEuc.head()

In [ ]:
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames


In [ ]:
newDistMat10binaryEuc=editSampleNames(newDistMat10binaryEuc)
newDistMat10binaryEuc.head()

In [ ]:
distanceVector=pd.DataFrame()
df=distMat_PNP434_sharingMatrixMoreThan10_euclidean
for i in range(434):
    if i%10==0:
        print i
    for j in range(i+1,434):
        sample1=df.index[i]
        sample2=df.columns.values[j]
        distance=df.iloc[i,j]
        distanceVector.loc[i,'sample1']=sample1
        distanceVector.loc[i,'sample2']=sample2
        distanceVector.loc[i,'distance']=distance
distanceVector.sort_values(by='distance')

In [ ]:
fig, axes = plt.subplots(figsize=(5,5))
axes.imshow(newDistMat10binaryEuc,cmap='PiYG')
#interpolation=None, vmin=0, vmax=0.01, cmap='PuRd'
plt.show()

## RA:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis'
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis=pd.read_pickle(file1)

In [ ]:
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.head()

In [ ]:
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.shape

In [ ]:
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis=distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.reindex_axis(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.sum().sort_values().index, axis=1)
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis=distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.reindex_axis(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.sum().sort_values().index, axis=0)
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.head()

In [ ]:
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames


In [ ]:
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis=editSampleNames(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis)
distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.head()

In [ ]:
distanceVector=pd.DataFrame()
df=distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis
for i in range(434):
    if i%10==0:
        print i
    for j in range(i+1,434):
        sample1=df.index[i]
        sample2=df.columns.values[j]
        distance=df.iloc[i,j]
        distanceVector.loc[i,'sample1']=sample1
        distanceVector.loc[i,'sample2']=sample2
        distanceVector.loc[i,'distance']=distance
distanceVector.sort_values(by='distance')

In [ ]:
fig, axes = plt.subplots(figsize=(5,5))
axes.imshow(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis,cmap='PiYG',vmin=0.88,vmax=1)
#interpolation=None, vmin=0, vmax=0.01, cmap='PuRd'
plt.show()

# more than 50

## binary:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan50_euclidean'
distMat_PNP434_sharingMatrixMoreThan50_euclidean=pd.read_pickle(file1)


In [ ]:
distMat_PNP434_sharingMatrixMoreThan50_euclidean.head()

In [ ]:
from skbio.stats.distance import DistanceMatrix


In [ ]:
import pandas as pd
from scipy.spatial import distance_matrix

In [ ]:
df=distMat_PNP434_sharingMatrixMoreThan50_euclidean
newDistMat50binaryEuc=pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)


In [ ]:
newDistMat50binaryEuc.shape

In [ ]:
newDistMat50binaryEuc=newDistMat50binaryEuc.reindex_axis(newDistMat50binaryEuc.sum().sort_values().index, axis=1)
newDistMat50binaryEuc=newDistMat50binaryEuc.reindex_axis(newDistMat50binaryEuc.sum().sort_values().index, axis=0)
newDistMat50binaryEuc.head()

In [ ]:
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames


In [ ]:
newDistMat50binaryEuc=editSampleNames(newDistMat50binaryEuc)
newDistMat50binaryEuc.head()

In [ ]:
distanceVector=pd.DataFrame()
df=distMat_PNP434_sharingMatrixMoreThan50_euclidean
for i in range(434):
    if i%10==0:
        print i
    for j in range(i+1,434):
        sample1=df.index[i]
        sample2=df.columns.values[j]
        distance=df.iloc[i,j]
        distanceVector.loc[i,'sample1']=sample1
        distanceVector.loc[i,'sample2']=sample2
        distanceVector.loc[i,'distance']=distance
distanceVector.sort_values(by='distance')

In [ ]:
fig, axes = plt.subplots(figsize=(5,5))
axes.imshow(newDistMat50binaryEuc,cmap='PiYG')
#interpolation=None, vmin=0, vmax=0.01, cmap='PuRd'
plt.show()

## RA:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis'
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis=pd.read_pickle(file1)


In [ ]:
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.head()

In [ ]:
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.shape

In [ ]:
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis=distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.reindex_axis(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.sum(axis=1).sort_values().index, axis=1)
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis=distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.reindex_axis(distMat_PNP434_sharingMatrixMoreThan10RA_braycurtis.sum(axis=0).sort_values().index, axis=0)
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.head()

In [ ]:
from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames


In [ ]:
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis=editSampleNames(distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis)
distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis.head()

In [ ]:
distanceVector=pd.DataFrame()
df=distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis
count=0
for i in range(434):
    if i%10==0:
        print i
    for j in range(i+1,434):
#         print i,j
        sample1=df.index[i]
        sample2=df.columns.values[j]
        distance=df.iloc[i,j]
#         print sample1,sample2,distance
        distanceVector.loc[count,'sample1']=sample1
        distanceVector.loc[count,'sample2']=sample2
        distanceVector.loc[count,'distance']=distance
        count=count+1
distanceVector.sort_values(by='distance')

In [ ]:
fig, axes = plt.subplots(figsize=(5,5))
axes.imshow(distMat_PNP434_sharingMatrixMoreThan50RA_braycurtis,cmap='PiYG',vmin=0.75,vmax=1)
#interpolation=None, vmin=0, vmax=0.01, cmap='PuRd'
plt.show()

## define a function that order sample pairs according to distance in a distance matrix:

In [ ]:
distMatNameList=['50_euclidean','50RA_braycurtis','1Binary_euclidean','1RA_braycurtis',
                '10_euclidean','10RA_braycurtis']

for n, distMatName in enumerate(distMatNameList):
    distMatFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan%s' %distMatName  
    distMat=pd.read_pickle(distMatFile)
    
    print n, distMatName
    order_sample_pairs_by_distance(distMat,distMatName)

In [ ]:
distMatNameList=['50_euclidean','50RA_braycurtis','1Binary_euclidean','1RA_braycurtis',
                '10_euclidean','10RA_braycurtis']

for n, distMatName in enumerate(distMatNameList):
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/samplePairOrderByDistance/distOrder_%s' %distMatName
    distanceVector=pd.read_pickle(file1)
    print distMatName
    distanceVector=distanceVector.sort_values(by='distance')
    print 'distanceVector shape is %s_%s' %(distanceVector.shape[0],distanceVector.shape[1])
    print distanceVector.head()
    print distanceVector.tail()

### order jaccard distMats - 1.2.18:

In [3]:
distMatNameList=['50_jaccard','10_jaccard','1_jaccard','501PCsUsed_euclidean','101PCsUsed_euclidean','11PCsUsed_euclidean']

for n, distMatName in enumerate(distMatNameList):
    distMatFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/Features/distMat_PNP434_sharingMatrixMoreThan%s' %distMatName  
    distMat=pd.read_pickle(distMatFile)
    
    print n, distMatName
    distanceVector=order_sample_pairs_by_distance(distMat,distMatName)
                 

    distanceVector=distanceVector.sort_values(by='distance')
    print 'distanceVector shape is %s_%s' %(distanceVector.shape[0],distanceVector.shape[1])
    print distanceVector.head()
    print distanceVector.tail()

0 50_jaccard
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
distanceVector shape is 93961_3
      sample1 sample2  distance
84143   BD136   BD146  0.630042
77037   BD160   BD146  0.649930
27443    BD13   BD146  0.665284
90160   BD146   BD145  0.667614
4620    BD153   BD146  0.689900
      sample1 sample2  distance
55861   BD453   BD801     0.995
36130   BD484   BD328     1.000
54085   BD322   BD319     1.000
7024    BD640   BD328     1.000
7053    BD640   BD631     1.000
1 10_jaccard
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
distanceVector shape is 93961_3
      sample1 sample2  distance
6341    BD431   BD440  0.802136
34259   BD788   BD809  0.856856
83064   BD699   BD706  0.860415
84143   BD136   BD146  0.875674
11492   BD838   BD617  0.876740
    

### order microbiome distMats - 1.2.18:

In [ ]:
mb_distMat_list=['RA_braycurtis','binary_euclidean']

for n, distMatName in enumerate(mb_distMat_list):
    distMatFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/newPhenotypes/distMat_PNP434_Microbiome%s' %distMatName   
    distMat=pd.read_pickle(distMatFile)
    
    print n, distMatName
    distanceVector=order_sample_pairs_by_distance(distMat,distMatName)
                 

    distanceVector=distanceVector.sort_values(by='distance')
    print 'distanceVector shape is %s_%s' %(distanceVector.shape[0],distanceVector.shape[1])
    print distanceVector.head()
    print distanceVector.tail()

0 50_jaccard
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
distanceVector shape is 93961_3
      sample1 sample2  distance
84143   BD136   BD146  0.630042
77037   BD160   BD146  0.649930
27443    BD13   BD146  0.665284
90160   BD146   BD145  0.667614
4620    BD153   BD146  0.689900
      sample1 sample2  distance
55861   BD453   BD801     0.995
36130   BD484   BD328     1.000
54085   BD322   BD319     1.000
7024    BD640   BD328     1.000
7053    BD640   BD631     1.000
1 10_jaccard
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
distanceVector shape is 93961_3
      sample1 sample2  distance
6341    BD431   BD440  0.802136
34259   BD788   BD809  0.856856
83064   BD699   BD706  0.860415
84143   BD136   BD146  0.875674
11492   BD838   BD617  0.876740
    

In [ ]:
def count_common_seqs(sharedMat,nCutOff):
#     from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix
    
    nSamples=len(sharedMat.index)
    sharedMat=editSampleNames(sharedMat)

    commonSeqsCountDF=pd.DataFrame()
    count=0
    for i in range(nSamples):
#             if i%10==0:
#                 print i
            for j in range(i+1,nSamples):
                print i,j
                df=sharedMat.iloc[[i,j],:]
                dfSum=df.sum(axis=0)

                sample1=sharedMat.index[i]
                sample2=sharedMat.index[j]

                if 2 in dfSum.value_counts():
                    nCommonSeqs=dfSum.value_counts()[2]
                else:
                    nCommonSeqs=0
                commonSeqsCountDF.loc[count,'sample1']=sample1
                commonSeqsCountDF.loc[count,'sample2']=sample2
                commonSeqsCountDF.loc[count,'nCommonSeqs']=nCommonSeqs
                count=count+1
    commonSeqsCountDF=commonSeqsCountDF.sort_values(by='nCommonSeqs')
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/commonSeqsCountDFs/commonSeqsCount_%s' %nCutOff
    commonSeqsCountDF.to_pickle(file1)
    
    return commonSeqsCountDF

In [ ]:
nCutOffList=[50,10,1]
# nCutOffList=[50]
for n, nCutOff in enumerate(nCutOffList):
    sharedMatFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples' %nCutOff
    sharedMat=pd.read_pickle(sharedMatFile)
    
    print n, nCutOff
    count_common_seqs(sharedMat,nCutOff)

In [ ]:
def count_NonCommon_seqs(sharedMat,nCutOff):
#     from Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix
    
    nSamples=len(sharedMat.index)
    sharedMat=editSampleNames(sharedMat)

    NonCommonSeqsCountDF=pd.DataFrame()
    count=0
    for i in range(nSamples):
#             if i%10==0:
#                 print i
            for j in range(i+1,nSamples):
                print i,j
                df=sharedMat.iloc[[i,j],:]
                dfSum=df.sum(axis=0)

                sample1=sharedMat.index[i]
                sample2=sharedMat.index[j]

                if 2 in dfSum.value_counts():
                    NonCommonSeqs=dfSum.value_counts()[1]
                else:
                    NonCommonSeqs=0
                NonCommonSeqsCountDF.loc[count,'sample1']=sample1
                NonCommonSeqsCountDF.loc[count,'sample2']=sample2
                NonCommonSeqsCountDF.loc[count,'NonCommonSeqs']=NonCommonSeqs
                count=count+1
    NonCommonSeqsCountDF=NonCommonSeqsCountDF.sort_values(by='NonCommonSeqs')
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/commonSeqsCountDFs/NonCommonSeqsCount_%s' %nCutOff
    NonCommonSeqsCountDF.to_pickle(file1)
    
    return NonCommonSeqsCountDF

In [ ]:
nCutOffList=[50,10,1]
# nCutOffList=[50]
for n, nCutOff in enumerate(nCutOffList):
    sharedMatFile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PublicSeqAnalysis/sharingMatrix_moreThan%s_434Samples' %nCutOff
    sharedMat=pd.read_pickle(sharedMatFile)
    
    print n, nCutOff
    NonCommonSeqsCountDF=count_NonCommon_seqs(sharedMat,nCutOff)

In [ ]:
NonCommonSeqsCountDF